# Variational autoencoder
This notebook will investiga

In [64]:
%matplotlib
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from variational_autoencoder import VariationalAutoencoder

# For dimension reduction
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# for visualization and evaluation
from latent_plane_mosaic import LatentPlaneMosaic
from latent_interpolation_mosaic import LatentInterpolationMosaic
from sample_scatter_gui import SampleScatterGUI

Using matplotlib backend: Qt5Agg


In [65]:
# Global settings
model_path = "model_bc_rf_10_lf_1"
num_samples = 10000
np.set_printoptions(precision=0)

## Preprocess data


In [66]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255
mnist_labels = np.concatenate([y_train, y_test], axis=0)
input_shape = mnist_digits.shape[1:]
num_digits = len(mnist_labels)
sample_index = rnd.randint(0, num_digits, size = (num_samples,))
mnist_digits = mnist_digits[sample_index]
mnist_labels = mnist_labels[sample_index]

## Load Variational autoencoder

In [67]:
# Variational autoencoder
disc_model = keras.models.load_model(disc_path, 
                         custom_objects={"VariationalAutoencoder": VariationalAutoencoder})


### Print setup details

In [50]:
print("Input shape: ", input_shape)
print("Latent dim: ",  disc_model.latent_dim, cont_model.latent_dim)
print("Num parameters: ", disc_model.count_params(), cont_model.count_params())



Input shape:  (28, 28, 1)
Latent dim:  10 10
Num parameters:  368407 368407


## Reconstruct and dimension reductions

In [51]:
# Reconstructions
disc_mnist_distribution = disc_model.encoder(mnist_digits).numpy() 
disc_mnist_encoded = disc_mnist_distribution[:,0,:]
disc_mnist_log_variance = disc_mnist_distribution[:,1,:]

cont_mnist_distribution = cont_model.encoder(mnist_digits).numpy() 
cont_mnist_encoded = cont_mnist_distribution[:,0,:]
cont_mnist_log_variance = cont_mnist_distribution[:,1,:]

# t-SNE
disc_mnist_tsne2d = TSNE(n_components = 2).fit_transform(disc_mnist_encoded)
cont_mnist_tsne2d = TSNE(n_components = 2).fit_transform(cont_mnist_encoded)

#pca
disc_pca = PCA().fit(disc_mnist_encoded)
disc_mnist_pca = disc_pca.transform(disc_mnist_encoded)
disc_latent_cov = np.cov(disc_mnist_pca.T)
disc_latent_std = np.sqrt(disc_pca.explained_variance_)
disc_latent_axes = disc_pca.components_
disc_latent_mean = disc_pca.mean_

cont_pca = PCA().fit(cont_mnist_encoded)
cont_mnist_pca = cont_pca.transform(cont_mnist_encoded)
cont_latent_cov = np.cov(cont_mnist_pca.T)
cont_latent_std = np.sqrt(cont_pca.explained_variance_)
cont_latent_axes = cont_pca.components_
cont_latent_mean = cont_pca.mean_


In [52]:
disc_mnist_encoded.shape


(10000, 10)

## Investigation

### Show reconstructions

In [18]:
num_col = 4
num_row = 4
num_img = num_row*num_col
fig, axs = plt.subplots(num_row, num_col)

rec_index = rnd.randint(num_samples,size =(num_row*num_col,) )
digits = mnist_digits[rec_index]
reconstructions = disc_model(digits)
i = 0
for col in range(num_col):
    for row in range(num_row): 
        ax = axs[row, col]
        im = np.concatenate((digits[i],
                             reconstructions[i]),
                             axis = 1)
        ax.imshow(im)
        ax.set_title("Reconstruction")
        i = i +1

In [19]:
num_col = 4
num_row = 4
num_img = num_row*num_col
fig, axs = plt.subplots(num_row, num_col)

#rec_index = rnd.randint(num_samples,size =(num_row*num_col,) )
digits = mnist_digits[rec_index]
reconstructions = cont_model(digits)
i = 0
for col in range(num_col):
    for row in range(num_row): 
        ax = axs[row, col]
        im = np.concatenate((digits[i],
                             reconstructions[i]),
                             axis = 1)
        ax.imshow(im)
        ax.set_title("Reconstruction")
        i = i +1

### Latent

In [25]:
indeces = rnd.randint(num_samples,size = (3,))

ul = mnist_digits[indeces[0]]
ur = mnist_digits[indeces[1]]
dl = mnist_digits[indeces[2]]
z =  np.zeros(dl.shape)
corner_image = np.concatenate( (np.concatenate((ul,ur),axis = 1),
                                np.concatenate((dl,z ),axis = 1)), axis = 0)

mosaic = LatentInterpolationMosaic(
                          disc_model.encode,
                          disc_model.decoder,
                          mnist_digits,
                          indeces,
                          num_row = 15,
                          num_col = 15).mosaic





fig, axs = plt.subplots(1,2)
axs[0].imshow(corner_image)
axs[1].imshow(mosaic)

In [27]:
indeces = rnd.randint(num_samples,size = (3,))

ul = mnist_digits[indeces[0]]
ur = mnist_digits[indeces[1]]
dl = mnist_digits[indeces[2]]
z =  np.zeros(dl.shape)
corner_image = np.concatenate( (np.concatenate((ul,ur),axis = 1),
                                np.concatenate((dl,z ),axis = 1)), axis = 0)

mosaic = LatentInterpolationMosaic(
                          cont_model.encode,
                          cont_model.decoder,
                          mnist_digits,
                          indeces,
                          num_row = 15,
                          num_col = 15).mosaic





fig, axs = plt.subplots(1,2)
axs[0].imshow(corner_image)
axs[1].imshow(mosaic)

### Scatter

In [16]:
SampleScatterGUI(disc_mnist_tsne2d, mnist_labels, mnist_digits)
#if latent dim == 2
#SampleScatterGUI(mnist_encoded, mnist_labels, mnist_digits)

In [17]:
SampleScatterGUI(cont_mnist_tsne2d, mnist_labels, mnist_digits)
#if latent dim == 2
#SampleScatterGUI(mnist_encoded, mnist_labels, mnist_digits)

### Spread

In [59]:
fig, ax = plt.subplots(1,1)
ax.plot(disc_latent_std, 'o--')
ax.set_yscale("log")
ax.set_xlabel("Principal axis index in decreasing order")
ax.set_ylabel("Principal variance for the corresponding axis")
ax.set_title("Principal variance of Discrete network")

Text(0.5, 1.0, 'Principal variance of Discrete network')

In [60]:
fig, ax = plt.subplots(1,1)
ax.plot(cont_latent_std, 'o--')
ax.set_yscale("log")
ax.set_xlabel("Principal axis index in decreasing order")
ax.set_ylabel("Principal variance for the corresponding axis")
ax.set_title("Principal variance of continues network")

Text(0.5, 1.0, 'Principal variance of continues network')

In [61]:
c = disc_latent_cov-np.diag(np.diag(disc_latent_cov))
print("Max non-diagonal covariance ", np.max(np.abs(c)))

Max non-diagonal covariance  3.2063664843663787e-07


In [62]:
c = cont_latent_cov-np.diag(np.diag(cont_latent_cov))
print("Max non-diagonal covariance ", np.max(np.abs(c)))

Max non-diagonal covariance  7.247497918139246e-07


Here we have that the first 6 principal axises are almost one and the remaining 4 are almost zero (e-3). This is combination with that the cross covariance seem to be super much zero (e-15), indicates that we have a six dimesnioal sphere in this ten dimesional latent space. Interesting to see is also if you create a latent vector in the pca-vector space. Then comparing the latent value in each dimension times the spread in each dimension, gives you the importance of that dimension in that latent point. A low importance will not have an effect on the output (kind of like a low derivative...), while a hight value will give a big difference.

In [63]:
axes_index = [2,3]
latent_vectors = disc_latent_axes[axes_index]
latent_origin = disc_latent_mean
scaling_factors = np.array([5,5])

mosaic = LatentPlaneMosaic(disc_model.decoder,
                  latent_vectors = (scaling_factors*latent_vectors.T).T,
                  latent_origin = latent_origin,
                  num_row = 20,
                  num_col = 20).mosaic

fig,ax = plt.subplots(1,1)
ax.imshow(mosaic)


In [22]:
axes_index = [2,3]
latent_vectors = cont_latent_axes[axes_index]
latent_origin = cont_latent_mean
scaling_factors = np.array([5,5])

mosaic = LatentPlaneMosaic(cont_model.decoder,
                  latent_vectors = (scaling_factors*latent_vectors.T).T,
                  latent_origin = latent_origin,
                  num_row = 20,
                  num_col = 20).mosaic

fig,ax = plt.subplots(1,1)
ax.imshow(mosaic)

In [21]:
latent_origin

array([ 0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        0.,  0.,  0.], dtype=float32)

## Spred 

In [53]:
disc_dist = np.linalg.norm(disc_mnist_encoded, axis = 1)
disc_std = np.exp(disc_mnist_log_variance.flatten())

cont_dist = np.linalg.norm(cont_mnist_encoded, axis = 1)
cont_std = np.exp(cont_mnist_log_variance.flatten())

In [58]:
nbins = 100
fig, axs = plt.subplots(2,2)
axs[0,0].hist(disc_dist, nbins)
axs[0,1].hist(disc_std, nbins)
axs[1,0].hist(cont_dist, nbins)
axs[1,1].hist(cont_std, nbins)

(array([6.e+03, 1.e+04, 1.e+04, 1.e+04, 1.e+04, 7.e+03, 6.e+03, 5.e+03,
        4.e+03, 3.e+03, 3.e+03, 2.e+03, 2.e+03, 2.e+03, 1.e+03, 1.e+03,
        9.e+02, 8.e+02, 7.e+02, 5.e+02, 5.e+02, 4.e+02, 4.e+02, 3.e+02,
        3.e+02, 2.e+02, 2.e+02, 2.e+02, 2.e+02, 2.e+02, 1.e+02, 1.e+02,
        1.e+02, 1.e+02, 1.e+02, 1.e+02, 1.e+02, 1.e+02, 1.e+02, 9.e+01,
        8.e+01, 6.e+01, 6.e+01, 6.e+01, 7.e+01, 5.e+01, 6.e+01, 5.e+01,
        6.e+01, 5.e+01, 5.e+01, 4.e+01, 5.e+01, 4.e+01, 4.e+01, 4.e+01,
        3.e+01, 3.e+01, 2.e+01, 3.e+01, 2.e+01, 2.e+01, 2.e+01, 2.e+01,
        2.e+01, 1.e+01, 2.e+01, 1.e+01, 1.e+01, 2.e+01, 8.e+00, 7.e+00,
        1.e+01, 1.e+01, 1.e+01, 2.e+01, 1.e+01, 1.e+01, 1.e+01, 7.e+00,
        7.e+00, 9.e+00, 7.e+00, 4.e+00, 7.e+00, 4.e+00, 2.e+00, 4.e+00,
        9.e+00, 2.e+00, 2.e+00, 4.e+00, 1.e+00, 2.e+00, 1.e+00, 1.e+00,
        0.e+00, 0.e+00, 0.e+00, 2.e+00]),
 array([3.e-05, 2.e-04, 5.e-04, 7.e-04, 9.e-04, 1.e-03, 1.e-03, 2.e-03,
        2.e-03, 2.e-03

In [65]:
model.summary()

Model: "variational_autoencoder_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder (Sequential)         (None, 2, 8)              210580    
_________________________________________________________________
Decoder (Sequential)         (None, 28, 28, 1)         157017    
_________________________________________________________________
normal_sampling_layer_3 (Nor multiple                  0         
Total params: 367,605
Trainable params: 367,597
Non-trainable params: 8
_________________________________________________________________


In [23]:
disc_model.summary()



Model: "variational_autoencoder_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder (Sequential)         (None, 2, 10)             210984    
_________________________________________________________________
Decoder (Sequential)         (None, 28, 28, 1)         157417    
_________________________________________________________________
normal_sampling_layer (Norma multiple                  0         
Total params: 368,407
Trainable params: 368,401
Non-trainable params: 6
_________________________________________________________________
